In [ ]:
# requirements
# install flipside

# 3 chains : base, optimism, blast

In [2]:
# !pip install flipside

In [7]:
from flipside import Flipside
from dotenv import load_dotenv
import os

load_dotenv(".env")

FLIPSIDE_API_KEY = os.environ.get("FLIPSIDE_API_KEY")

# Initialize `Flipside` with your API Key and API Url
flipside = Flipside(FLIPSIDE_API_KEY, "https://api-v2.flipsidecrypto.xyz")


#set it according to your needs.
DATE_CUTT_OFF = "2024-09-01"
ETH_VALUE=2525.86

# an active user is defined X transactions over Y days
X = 5
Y = 60

# Dex

In [8]:
# DEX metrics

def dex_num_tokens(blockchain, contract_address):
    
    #get the unique number of tokens supported on given chain by a dex
    sql = f"""
            SELECT COUNT(DISTINCT t.value) AS unique_token_options
            FROM {blockchain}.defi.dim_dex_liquidity_pools,
                 LATERAL FLATTEN(input => tokens) t
            WHERE factory_address = '{contract_address}'
            AND creation_time < {DATE_CUTT_OFF};
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])


    
def dex_nusers(blockchain, contract_address):
    
    #get the number of active users on given chain by a dex
    sql = f"""
           WITH contract_addresses AS (
                SELECT DISTINCT address
                FROM {blockchain}.core.dim_contracts
                WHERE created_block_timestamp < {DATE_CUTT_OFF}
            ),
            user_transactions AS (
                SELECT 
                    t.from_address,
                    t.block_timestamp,
                    COUNT(*) OVER (
                        PARTITION BY t.from_address 
                        ORDER BY t.block_timestamp 
                        RANGE BETWEEN INTERVAL {Y} DAY PRECEDING AND CURRENT ROW
                    ) AS tx_count_last_Y_days
                FROM {blockchain}.core.fact_transactions t
                LEFT JOIN contract_addresses c ON t.from_address = c.address
                WHERE t.to_address = {contract_address}
                AND t.block_timestamp < {DATE_CUTT_OFF}
                AND c.address IS NULL
            ),
            active_users AS (
                SELECT DISTINCT from_address
                FROM user_transactions
                WHERE tx_count_last_Y_days >= {X}
            )
            SELECT COUNT(*) AS active_users
            FROM active_users;
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def dex_ntrxns(blockchain, contract_address):
    
    #get the number of transactions on given chain by a dex
    sql = f"""
            SELECT COUNT(*) AS total_transactions
            FROM {blockchain}.core.fact_transactions
            WHERE to_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF};
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def dex_sumtrxns(blockchain, contract_address):
    
    #get the value of total transactions in eth. 
    sql = f"""
            SELECT SUM(value) AS total_eth_value
            FROM {blockchain}.core.fact_transactions
            WHERE to_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF};
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def dex_tvl(blockchain, name):
    
    sql = f"""
    
    
    """
    
    return()



In [9]:
#bridge metrics

def bridge_nchains(blockchain, contract_address):
    
    #get the number of chains supported by a bridge
    
    sql = f"""
        SELECT COUNT(DISTINCT destination_chain) AS unique_chains
        FROM {blockchain}.defi.ez_bridge_activity
        where bridge_address = {contract_address}
        AND block_timestamp < {DATE_CUTT_OFF};
    """
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def bridge_nusers(blockchain, contract_address):
    
    #get the number of active users
    
    sql = f"""
           WITH contract_addresses AS (
                SELECT DISTINCT address
                FROM {blockchain}.core.dim_contracts
                WHERE created_block_timestamp < {DATE_CUTT_OFF}
            ),
            user_transactions AS (
                SELECT 
                    t.origin_from_address,
                    t.block_timestamp,
                    COUNT(*) OVER (
                        PARTITION BY t.origin_from_address 
                        ORDER BY t.block_timestamp 
                        RANGE BETWEEN INTERVAL {Y} DAY PRECEDING AND CURRENT ROW
                    ) AS tx_count_last_Y_days
                FROM {blockchain}.core.fact_transactions t
                LEFT JOIN contract_addresses c ON t.origin_from_address = c.address
                WHERE t.bridge_address = {contract_address}
                AND t.block_timestamp < {DATE_CUTT_OFF}
            ),
            active_users AS (
                SELECT DISTINCT from_address
                FROM user_transactions
                WHERE tx_count_last_Y_days >= {X}
            )
            SELECT COUNT(*) AS active_users
            FROM active_users;
    """
    
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def bridge_nbtrxns(blockchain, contract_address):
    
    #get the number of bridge transactions
    
    sql = f"""
        SELECT COUNT(*) AS total_bridge_transactions
        FROM {blockchain}.defi.ez_bridge_activity
        where bridge_address = {contract_address}
        AND block_timestamp < {DATE_CUTT_OFF};
    """
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])



def bridge_valtrxns(blockchain, contract_address):
    
    #get the total eth value of transactions. We will get total value of transactions in usd and convert it to eth
    # based on eth price on cutt off date which is 2525.86 on 31-aug-2024
    # change the metric with adding eth price of each day then calculating total eth each day
    sql = f"""
        WITH hourly_eth_prices AS (
            SELECT hour, price AS eth_price
            FROM {blockchain}.price.ez_prices_hourly
            WHERE symbol = 'ETH'
            AND hour < {DATE_CUTT_OFF}
        ),
        bridge_transactions AS(
            SELECT block_timestamp, amount_usd
            FROM {blockchain}.defi.ez_bridge_activity
            where bridge_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF};

        ),
        transactions_with_prices AS (
            SELECT 
                b.block_timestamp,
                b.amount_usd,
                h.eth_price,
                b.amount_usd / h.eth_price AS eth_value
            FROM bridge_transactions b
            JOIN hourly_eth_prices h ON DATE_TRUNC('hour', b.block_timestamp) = h.hour
        )
        SELECT 
            SUM(eth_value) AS total_eth_value
        FROM transactions_with_prices;
    """
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def bridge_medianfee(blockchain, contract_address):
    
    
    sql = f"""
        WITH bridge_fees AS (
            SELECT 
                b.tx_hash,
                t.tx_fee AS eth_cost
            FROM {blockchain}.defi.ez_bridge_activity b
            JOIN {blockchain}.core.fact_transactions t ON b.tx_hash = t.tx_hash
            WHERE b.block_timestamp < {DATE_CUTT_OFF}
            AND   b.bridge_address =  {contract_address}
        )
        SELECT 
            PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY eth_cost) AS median_eth_cost
        FROM bridge_fees;

    """
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])



In [10]:
## Lending pools

def lending_nproviders(blockchain, contract_address):
    
    #get the number of collateral providers
    
    sql = f"""
        SELECT COUNT(DISTINCT depositor) AS unique_collateral_providers
        FROM {blockchain}.defi.ez_lending_deposits
        where contract_address = {contract_address}
        AND block_timestamp < {DATE_CUTT_OFF};
    """
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def lending_nborrowers(blockchain, contract_address):
    
    #get the number of active borrowers
    
    sql = f"""
        SELECT COUNT(DISTINCT borrower) AS active_borrowers
        FROM {blockchain}.defi.ez_lending_borrows
        where contract_address = {contract_address}
        AND block_timestamp < {DATE_CUTT_OFF};
    """
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def lending_ntrxns(blockchain, contract_address):
    
    #get the number of transactions
    
    sql = f"""
        SELECT 
            (SELECT COUNT(*) FROM {blockchain}.defi.ez_lending_deposits where contract_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF} +
            (SELECT COUNT(*) FROM {blockchain}.defi.ez_lending_borrows where contract_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF}; +
            (SELECT COUNT(*) FROM {blockchain}.defi.ez_lending_repayments where contract_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF}; +
            (SELECT COUNT(*) FROM {blockchain}.defi.ez_lending_withdraws where contract_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF}; +
            (SELECT COUNT(*) FROM {blockchain}.defi.ez_lending_liquidations where contract_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF}; +
            (SELECT COUNT(*) FROM {blockchain}.defi.ez_lending_flashloans where contract_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF};
        AS total_transactions;
    """
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])


def lending_valtrxns(blockchain, contract_address):
    
    # get the total value of transactions
    # query needs to be improved.
    
    sql = f"""
    
            WITH hourly_eth_prices AS (
            SELECT hour, price AS eth_price
            FROM {blockchain}.price.ez_prices_hourly
            WHERE symbol = 'ETH'
            AND hour < {DATE_CUTT_OFF}
        ),
        lending_transactions AS (
            SELECT block_timestamp, amount_usd
            FROM (
                SELECT block_timestamp, amount_usd FROM {blockchain}.defi.ez_lending_deposits
                where contract_address = {contract_address} AND block_timestamp < {DATE_CUTT_OFF}
                UNION ALL
                SELECT block_timestamp, amount_usd FROM {blockchain}.defi.ez_lending_borrows 
                where contract_address = {contract_address} AND block_timestamp < {DATE_CUTT_OFF}
                UNION ALL
                SELECT block_timestamp, amount_usd FROM {blockchain}.defi.ez_lending_repayments
                where contract_address = {contract_address} AND block_timestamp < {DATE_CUTT_OFF}
                UNION ALL
                SELECT block_timestamp, amount_usd FROM {blockchain}.defi.ez_lending_withdraws
                where contract_address = {contract_address} AND block_timestamp < {DATE_CUTT_OFF}
                UNION ALL
                SELECT block_timestamp, amount_usd FROM {blockchain}.defi.ez_lending_liquidations
                where contract_address = {contract_address} AND block_timestamp < {DATE_CUTT_OFF}
                UNION ALL
                SELECT block_timestamp, flashloan_amount_usd AS amount_usd FROM {blockchain}.defi.ez_lending_flashloans
                where contract_address = {contract_address} AND block_timestamp < {DATE_CUTT_OFF}
            ) AS all_transactions
            
        ),
        transactions_with_prices AS (
            SELECT 
                l.block_timestamp,
                l.amount_usd,
                h.eth_price,
                l.amount_usd / h.eth_price AS eth_value
            FROM lending_transactions l
            JOIN hourly_eth_prices h ON DATE_TRUNC('hour', l.block_timestamp) = h.hour
        )
        SELECT 
            SUM(eth_value) AS total_eth_value
        FROM transactions_with_prices;
    """
    
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])


In [11]:
## RWA

def rwa_ntokens(blockchain, contract_address):
    
    return(0)


def rwa_nusers(blockchain, contract_address):
    
    #get the number of active users on given chain by a RWA
    sql = f"""
           WITH contract_addresses AS (
                SELECT DISTINCT address
                FROM {blockchain}.core.dim_contracts
                WHERE created_block_timestamp < {DATE_CUTT_OFF}
            ),
            user_transactions AS (
                SELECT 
                    t.from_address,
                    t.block_timestamp,
                    COUNT(*) OVER (
                        PARTITION BY t.from_address 
                        ORDER BY t.block_timestamp 
                        RANGE BETWEEN INTERVAL {Y} DAY PRECEDING AND CURRENT ROW
                    ) AS tx_count_last_Y_days
                FROM {blockchain}.core.fact_transactions t
                LEFT JOIN contract_addresses c ON t.from_address = c.address
                WHERE t.to_address = {contract_address}
                AND t.block_timestamp < {DATE_CUTT_OFF}
                AND c.address IS NULL
            ),
            active_users AS (
                SELECT DISTINCT from_address
                FROM user_transactions
                WHERE tx_count_last_Y_days >= {X}
            )
            SELECT COUNT(*) AS active_users
            FROM active_users;
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def rwa_ntrxns(blockchain, contract_address):
    
    #get the number of transactions on given chain by a dex
    sql = f"""
            SELECT COUNT(*) AS total_transactions
            FROM {blockchain}.core.fact_transactions
            WHERE to_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF};
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def rwa_sumtrxns(blockchain, contract_address):
    
    #get the value of total transactions in eth. 
    sql = f"""
            SELECT SUM(value) AS total_eth_value
            FROM {blockchain}.core.fact_transactions
            WHERE to_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF};
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])


def rwa_tvl(blockchain, contract_address):
    
    return()

In [12]:
## Staking

def staking_nusers(blockchain, contract_address):
    
    #get the number of active users on given chain by a dex
    sql = f"""
           WITH contract_addresses AS (
                SELECT DISTINCT address
                FROM {blockchain}.core.dim_contracts
                WHERE created_block_timestamp < {DATE_CUTT_OFF}
            ),
            user_transactions AS (
                SELECT 
                    t.from_address,
                    t.block_timestamp,
                    COUNT(*) OVER (
                        PARTITION BY t.from_address 
                        ORDER BY t.block_timestamp 
                        RANGE BETWEEN INTERVAL {Y} DAY PRECEDING AND CURRENT ROW
                    ) AS tx_count_last_Y_days
                FROM {blockchain}.core.fact_transactions t
                LEFT JOIN contract_addresses c ON t.from_address = c.address
                WHERE t.to_address = {contract_address}
                AND t.block_timestamp < {DATE_CUTT_OFF}
                AND c.address IS NULL
            ),
            active_users AS (
                SELECT DISTINCT from_address
                FROM user_transactions
                WHERE tx_count_last_Y_days >= {X}
            )
            SELECT COUNT(*) AS active_users
            FROM active_users;
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])


def staking_ntrxns(blockchain, contract_address):
    
    #get the number of transactions on given chain by a dex
    sql = f"""
            SELECT COUNT(*) AS total_transactions
            FROM {blockchain}.core.fact_transactions
            WHERE to_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF};
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def staking_sumtrxns(blockchain, contract_address):
    
    #get the value of total transactions in eth. 
    sql = f"""
            SELECT SUM(value) AS total_eth_value
            FROM {blockchain}.core.fact_transactions
            WHERE to_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF};
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    
    return(query_result_set.rows[0][0])


def staking_tvl(blockchain, contract_address):
    
    return()


# Creator

In [13]:
## Art Marketplaces and Other Media Marketplace


def artmarketplace_nac(blockchain, contract_address):
    
    # get the number of active creators
    sql = f"""
            WITH nft_contracts AS (
                SELECT DISTINCT nft_address
                FROM {blockchain}.nft.ez_nft_transfers
                where WHERE platform_address = {contract_address}
                AND block_timestamp < {DATE_CUTT_OFF}
            )
            SELECT 
                COUNT( DISTINCT dc.creator_address)
            FROM nft_contracts nc
            JOIN {blockchain}.core.dim_contracts dc ON nc.nft_address = dc.address
            WHERE dc.created_block_timestamp < {DATE_CUTT_OFF};
    """
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

# Note: We don't have direct minting information in the provided schema.
# This query assumes that the first transfer from the contract address is a mint.

def artmarketplace_nam(blockchain, platform_address):
    
    # get the number of active minters
    sql = f"""
            SELECT COUNT(DISTINCT nft_to_address) AS active_minters
            FROM {blockchain}.nft.ez_nft_transfers
            WHERE platform_address = {contract_address}
            AND nft_from_address = nft_address
            AND block_timestamp < {DATE_CUTT_OFF};
    """
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def artmarketplace_ntrxns(blockchain, contract_address):
    
    # get the number of transactions
    sql = f"""
            SELECT COUNT(*) AS marketplace_transactions
            FROM {blockchain}.nft.ez_nft_sales
            WHERE platform_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF};
    """
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])


def artmarketplace_valtrxns(blockchain, contract_address):
    
    # get the number of transactions
    sql = f"""
            WITH hourly_eth_prices AS (
                SELECT hour, price AS eth_price
                FROM {blockchain}.price.ez_prices_hourly
                WHERE symbol = 'ETH'
                    AND hour < {DATE_CUTT_OFF}
                ),
                nftmarketplace_transactions AS(
                    SELECT block_timestamp, total_fees_usd
                    FROM {blockchain}.nft.ez_nft_sales
                    WHERE platform_address = {contract_address}
                    AND block_timestamp < {DATE_CUTT_OFF};

                ),
                transactions_with_prices AS (
                    SELECT 
                        b.block_timestamp,
                        b.total_fees_usd,
                        h.eth_price,
                        b.amount_usd / h.eth_price AS eth_value
                    FROM nftmarketplace_transactions b
                    JOIN hourly_eth_prices h ON DATE_TRUNC('hour', b.block_timestamp) = h.hour
                )
                
            SELECT SUM(eth_value) AS total_eth_value
            FROM transactions_with_prices;
            
    """
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])



In [14]:
## Art NFTs
## Other Media NFTs

# assumes that drops mean different types of collections.
def artnft_ndrops(blockchain, contract_address):
    
    sql = f"""
        WITH nft_contracts AS (
            SELECT DISTINCT nft_address
            FROM {blockchain}.nft.ez_nft_transfers
            WHERE block_timestamp < {DATE_CUTT_OFF}
        )
        SELECT 
            COUNT(*) AS number_of_drops
        FROM nft_contracts nc
        JOIN {blockchain}.core.dim_contracts dc ON nc.nft_address = dc.address
        WHERE dc.created_block_timestamp < {DATE_CUTT_OFF};
    
    """
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])
    

def artnft_nam(blockchain, wallet_address):
    
    # get the number of active minters
    sql = f"""
            WITH creator_contracts AS (
                -- Identify contracts created by the minting wallet
                SELECT address AS nft_address
                FROM {blockchain}.core.dim_contracts
                WHERE creator_address = {wallet_address}
                AND created_block_timestamp < {DATE_CUTT_OFF}
            ),
            minting_events AS (
                -- Identify minting events for the creator's contracts
                SELECT DISTINCT nft_to_address AS minter
                FROM nft.ez_nft_transfers
                WHERE nft_address IN (SELECT nft_address FROM creator_contracts)
                AND (
                    nft_from_address = '0x0000000000000000000000000000000000000000'
                    OR nft_from_address = nft_address
                )
                AND block_timestamp < '2024-08-31'
            )
            SELECT COUNT( DISTINCT minter )
            FROM minting_events;

    """
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])


def artnft_valtrxns(blockchain, contract_address):
    
    # get the number of transactions
    sql = f"""
            WITH hourly_eth_prices AS (
                SELECT hour, price AS eth_price
                FROM {blockchain}.price.ez_prices_hourly
                WHERE symbol = 'ETH'
                    AND hour < {DATE_CUTT_OFF}
                ),
                nftmarketplace_transactions AS(
                    SELECT block_timestamp, total_fees_usd
                    FROM {blockchain}.nft.ez_nft_sales
                    WHERE nft_address = {contract_address}
                    AND block_timestamp < {DATE_CUTT_OFF};

                ),
                transactions_with_prices AS (
                    SELECT 
                        b.block_timestamp,
                        b.total_fees_usd,
                        h.eth_price,
                        b.amount_usd / h.eth_price AS eth_value
                    FROM nftmarketplace_transactions b
                    JOIN hourly_eth_prices h ON DATE_TRUNC('hour', b.block_timestamp) = h.hour
                )
                
            SELECT SUM(eth_value) AS total_eth_value
            FROM transactions_with_prices;
            
    """
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])



# Social

In [15]:
## Community Creation
def cc_nusers(blockchain, contract_address):
    
    #get the number of active users on given chain
    sql = f"""
            WITH contract_addresses AS (
                SELECT DISTINCT address
                FROM {blockchain}.core.dim_contracts
                WHERE created_block_timestamp < {DATE_CUTT_OFF}
            ),
            user_transactions AS (
                SELECT 
                    t.from_address,
                    t.block_timestamp,
                    COUNT(*) OVER (
                        PARTITION BY t.from_address 
                        ORDER BY t.block_timestamp 
                        RANGE BETWEEN INTERVAL {Y} DAY PRECEDING AND CURRENT ROW
                    ) AS tx_count_last_Y_days
                FROM {blockchain}.core.fact_transactions t
                LEFT JOIN contract_addresses c ON t.from_address = c.address
                WHERE t.to_address = {contract_address}
                AND t.block_timestamp < {DATE_CUTT_OFF}
                AND c.address IS NULL
            ),
            active_users AS (
                SELECT DISTINCT from_address
                FROM user_transactions
                WHERE tx_count_last_Y_days >= {X}
            )
            SELECT COUNT(*) AS active_users
            FROM active_users;
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def cc_fnusers(blockchain, contract_address):
    
    #get the number of active users on given chain which are also active on farcaster 
    #update the query with farcaster data
    sql = f"""
            WITH contract_addresses AS (
                SELECT DISTINCT address
                FROM {blockchain}.core.dim_contracts
                WHERE created_block_timestamp < {DATE_CUTT_OFF}
            ),
            user_transactions AS (
                SELECT 
                    t.from_address,
                    t.block_timestamp,
                    COUNT(*) OVER (
                        PARTITION BY t.from_address 
                        ORDER BY t.block_timestamp 
                        RANGE BETWEEN INTERVAL {Y} DAY PRECEDING AND CURRENT ROW
                    ) AS tx_count_last_Y_days
                FROM {blockchain}.core.fact_transactions t
                LEFT JOIN contract_addresses c ON t.from_address = c.address
                WHERE t.to_address = {contract_address}
                AND t.block_timestamp < {DATE_CUTT_OFF}
                AND c.address IS NULL
            ),
            active_users AS (
                SELECT DISTINCT from_address
                FROM user_transactions
                WHERE tx_count_last_Y_days >= {X}
            )
            SELECT count(DISTINCT AU.active_user)
            FROM AU
            JOIN farcasters f
            WHERE ARRAY_CONTAINS(AU.active_user::VARIANT, f.verified_addresses)
            
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

In [16]:
## Gaming 
def gaming_nusers(blockchain, contract_address):
    
    #get the number of active users on given chain
    sql = f"""
           WITH contract_addresses AS (
                SELECT DISTINCT address
                FROM {blockchain}.core.dim_contracts
                WHERE created_block_timestamp < {DATE_CUTT_OFF}
            ),
            user_transactions AS (
                SELECT 
                    t.from_address,
                    t.block_timestamp,
                    COUNT(*) OVER (
                        PARTITION BY t.from_address 
                        ORDER BY t.block_timestamp 
                        RANGE BETWEEN INTERVAL {Y} DAY PRECEDING AND CURRENT ROW
                    ) AS tx_count_last_Y_days
                FROM {blockchain}.core.fact_transactions t
                LEFT JOIN contract_addresses c ON t.from_address = c.address
                WHERE t.to_address = {contract_address}
                AND t.block_timestamp < {DATE_CUTT_OFF}
                AND c.address IS NULL
            ),
            active_users AS (
                SELECT DISTINCT from_address
                FROM user_transactions
                WHERE tx_count_last_Y_days >= {X}
            )
            SELECT COUNT(*) AS active_users
            FROM active_users;
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def gaming_fnusers(blockchain, contract_address):
    
    #get the number of active users on given chain which are also active on farcaster 
    #update the query with farcaster data and run active query on farcaster data not on transactions
    sql = f"""
            WITH contract_addresses AS (
                SELECT DISTINCT address
                FROM {blockchain}.core.dim_contracts
                WHERE created_block_timestamp < {DATE_CUTT_OFF}
            ),
            user_transactions AS (
                SELECT 
                    t.from_address,
                    t.block_timestamp,
                    COUNT(*) OVER (
                        PARTITION BY t.from_address 
                        ORDER BY t.block_timestamp 
                        RANGE BETWEEN INTERVAL {Y} DAY PRECEDING AND CURRENT ROW
                    ) AS tx_count_last_Y_days
                FROM {blockchain}.core.fact_transactions t
                LEFT JOIN contract_addresses c ON t.from_address = c.address
                WHERE t.to_address = {contract_address}
                AND t.block_timestamp < {DATE_CUTT_OFF}
                AND c.address IS NULL
            ),
            active_users AS (
                SELECT DISTINCT from_address
                FROM user_transactions
                WHERE tx_count_last_Y_days >= {X}
            )
            SELECT count(DISTINCT AU.active_user)
            FROM AU
            JOIN farcasters f
            WHERE ARRAY_CONTAINS(AU.active_user::VARIANT, f.verified_addresses)
            
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

In [17]:
## Betting and Prediction Markets

def bp_nusers(blockchain, contract_address):
    
    #get the number of active users on given chain
    sql = f"""
           WITH contract_addresses AS (
                SELECT DISTINCT address
                FROM {blockchain}.core.dim_contracts
                WHERE created_block_timestamp < {DATE_CUTT_OFF}
            ),
            user_transactions AS (
                SELECT 
                    t.from_address,
                    t.block_timestamp,
                    COUNT(*) OVER (
                        PARTITION BY t.from_address 
                        ORDER BY t.block_timestamp 
                        RANGE BETWEEN INTERVAL {Y} DAY PRECEDING AND CURRENT ROW
                    ) AS tx_count_last_Y_days
                FROM {blockchain}.core.fact_transactions t
                LEFT JOIN contract_addresses c ON t.from_address = c.address
                WHERE t.to_address = {contract_address}
                AND t.block_timestamp < {DATE_CUTT_OFF}
                AND c.address IS NULL
            ),
            active_users AS (
                SELECT DISTINCT from_address
                FROM user_transactions
                WHERE tx_count_last_Y_days >= {X}
            )
            SELECT COUNT(*) AS active_users
            FROM active_users;
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def bp_ntrxns(blockchain, contract_address):
    
    #get the number of transactions on given chain
    sql = f"""
            SELECT COUNT(*) AS total_transactions
            FROM {blockchain}.core.fact_transactions
            WHERE to_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF};
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def bp_sumtrxns(blockchain, contract_address):
    
    #get the value of total transactions in eth. 
    sql = f"""
            SELECT SUM(value) AS total_eth_value
            FROM {blockchain}.core.fact_transactions
            WHERE to_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF};
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def bp_tvl():
    
    return()


In [18]:
## Meme Community

def meme_nusers(blockchain, contract_address):
    
    #get the number of active users on given chain
    sql = f"""
           WITH contract_addresses AS (
                SELECT DISTINCT address
                FROM {blockchain}.core.dim_contracts
                WHERE created_block_timestamp < {DATE_CUTT_OFF}
            ),
            user_transactions AS (
                SELECT 
                    t.from_address,
                    t.block_timestamp,
                    COUNT(*) OVER (
                        PARTITION BY t.from_address 
                        ORDER BY t.block_timestamp 
                        RANGE BETWEEN INTERVAL {Y} DAY PRECEDING AND CURRENT ROW
                    ) AS tx_count_last_Y_days
                FROM {blockchain}.core.fact_transactions t
                LEFT JOIN contract_addresses c ON t.from_address = c.address
                WHERE t.to_address = {contract_address}
                AND t.block_timestamp < {DATE_CUTT_OFF}
                AND c.address IS NULL
            ),
            active_users AS (
                SELECT DISTINCT from_address
                FROM user_transactions
                WHERE tx_count_last_Y_days >= {X}
            )
            SELECT COUNT(*) AS active_users
            FROM active_users;
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def meme_fnusers(blockchain, contract_address):
    
    #get the number of active users on given chain which are also active on farcaster 
    #update the query with farcaster data and run active query on farcaster data not on transactions
    sql = f"""
            WITH contract_addresses AS (
                SELECT DISTINCT address
                FROM {blockchain}.core.dim_contracts
                WHERE created_block_timestamp < {DATE_CUTT_OFF}
            ),
            user_transactions AS (
                SELECT 
                    t.from_address,
                    t.block_timestamp,
                    COUNT(*) OVER (
                        PARTITION BY t.from_address 
                        ORDER BY t.block_timestamp 
                        RANGE BETWEEN INTERVAL {Y} DAY PRECEDING AND CURRENT ROW
                    ) AS tx_count_last_Y_days
                FROM {blockchain}.core.fact_transactions t
                LEFT JOIN contract_addresses c ON t.from_address = c.address
                WHERE t.to_address = {contract_address}
                AND t.block_timestamp < {DATE_CUTT_OFF}
                AND c.address IS NULL
            ),
            active_users AS (
                SELECT DISTINCT from_address
                FROM user_transactions
                WHERE tx_count_last_Y_days >= {X}
            )
            SELECT count(DISTINCT AU.active_user)
            FROM AU
            JOIN farcasters f
            WHERE ARRAY_CONTAINS(AU.active_user::VARIANT, f.verified_addresses)
            
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def meme_ntrxns(blockchain, contract_address):
    
    #get the number of transactions on given chain
    sql = f"""
            SELECT COUNT(*) AS total_transactions
            FROM {blockchain}.core.fact_transactions
            WHERE to_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF};
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def meme_sumtrxns(blockchain, contract_address):
    
    #get the value of total transactions in eth. 
    sql = f"""
            SELECT SUM(value) AS total_eth_value
            FROM {blockchain}.core.fact_transactions
            WHERE to_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF};
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def meme_tvl():
    
    return()


# Utility

In [19]:
## Donation

def donation_nusers(blockchain, contract_address):
    
    #get the number of active users on given chain
    sql = f"""
           WITH contract_addresses AS (
                SELECT DISTINCT address
                FROM {blockchain}.core.dim_contracts
                WHERE created_block_timestamp < {DATE_CUTT_OFF}
            ),
            user_transactions AS (
                SELECT 
                    t.from_address,
                    t.block_timestamp,
                    COUNT(*) OVER (
                        PARTITION BY t.from_address 
                        ORDER BY t.block_timestamp 
                        RANGE BETWEEN INTERVAL {Y} DAY PRECEDING AND CURRENT ROW
                    ) AS tx_count_last_Y_days
                FROM {blockchain}.core.fact_transactions t
                LEFT JOIN contract_addresses c ON t.from_address = c.address
                WHERE t.to_address = {contract_address}
                AND t.block_timestamp < {DATE_CUTT_OFF}
                AND c.address IS NULL
            ),
            active_users AS (
                SELECT DISTINCT from_address
                FROM user_transactions
                WHERE tx_count_last_Y_days >= {X}
            )
            SELECT COUNT(*) AS active_users
            FROM active_users;
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def donation_ntrxns(blockchain, contract_address):
    
    #get the number of transactions on given chain
    sql = f"""
            SELECT COUNT(*) AS total_transactions
            FROM {blockchain}.core.fact_transactions
            WHERE to_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF};
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def donation_sumtrxns(blockchain, contract_address):
    
    #get the value of total transactions in eth. 
    sql = f"""
            SELECT SUM(value) AS total_eth_value
            FROM {blockchain}.core.fact_transactions
            WHERE to_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF};
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

In [20]:
## Airdrop

def airdrop_nusers(blockchain, contract_address):
    
    #get the number of active users on given chain
    sql = f"""
           WITH contract_addresses AS (
                SELECT DISTINCT address
                FROM {blockchain}.core.dim_contracts
                WHERE created_block_timestamp < {DATE_CUTT_OFF}
            ),
            user_transactions AS (
                SELECT 
                    t.from_address,
                    t.block_timestamp,
                    COUNT(*) OVER (
                        PARTITION BY t.from_address 
                        ORDER BY t.block_timestamp 
                        RANGE BETWEEN INTERVAL {Y} DAY PRECEDING AND CURRENT ROW
                    ) AS tx_count_last_Y_days
                FROM {blockchain}.core.fact_transactions t
                LEFT JOIN contract_addresses c ON t.from_address = c.address
                WHERE t.to_address = {contract_address}
                AND t.block_timestamp < {DATE_CUTT_OFF}
                AND c.address IS NULL
            ),
            active_users AS (
                SELECT DISTINCT from_address
                FROM user_transactions
                WHERE tx_count_last_Y_days >= {X}
            )
            SELECT COUNT(*) AS active_users
            FROM active_users;
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def airdrop_sumtrxns(blockchain, contract_address):
    
    #get the value of total transactions in eth. 
    sql = f"""
            SELECT SUM(value) AS total_eth_value
            FROM {blockchain}.core.fact_transactions
            WHERE to_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF};
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

In [21]:
## Payments

def payment_nusers(blockchain, contract_address):
    
    #get the number of active users on given chain
    sql = f"""
           WITH contract_addresses AS (
                SELECT DISTINCT address
                FROM {blockchain}.core.dim_contracts
                WHERE created_block_timestamp < {DATE_CUTT_OFF}
            ),
            user_transactions AS (
                SELECT 
                    t.from_address,
                    t.block_timestamp,
                    COUNT(*) OVER (
                        PARTITION BY t.from_address 
                        ORDER BY t.block_timestamp 
                        RANGE BETWEEN INTERVAL {Y} DAY PRECEDING AND CURRENT ROW
                    ) AS tx_count_last_Y_days
                FROM {blockchain}.core.fact_transactions t
                LEFT JOIN contract_addresses c ON t.from_address = c.address
                WHERE t.to_address = {contract_address}
                AND t.block_timestamp < {DATE_CUTT_OFF}
                AND c.address IS NULL
            ),
            active_users AS (
                SELECT DISTINCT from_address
                FROM user_transactions
                WHERE tx_count_last_Y_days >= {X}
            )
            SELECT COUNT(*) AS active_users
            FROM active_users;
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def payments_ntrxns(blockchain, contract_address):
    
    #get the number of transactions on given chain
    sql = f"""
            SELECT COUNT(*) AS total_transactions
            FROM {blockchain}.core.fact_transactions
            WHERE to_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF};
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def payment_sumtrxns(blockchain, contract_address):
    
    #get the value of total transactions in eth. 
    sql = f"""
            SELECT SUM(value) AS total_eth_value
            FROM {blockchain}.core.fact_transactions
            WHERE to_address = {contract_address}
            AND block_timestamp < {DATE_CUTT_OFF};
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

In [22]:
## Identity
def identity_nusers(blockchain, contract_address):
    
    #get the number of active users on given chain
    sql = f"""
           WITH contract_addresses AS (
                SELECT DISTINCT address
                FROM {blockchain}.core.dim_contracts
                WHERE created_block_timestamp < {DATE_CUTT_OFF}
            ),
            user_transactions AS (
                SELECT 
                    t.from_address,
                    t.block_timestamp,
                    COUNT(*) OVER (
                        PARTITION BY t.from_address 
                        ORDER BY t.block_timestamp 
                        RANGE BETWEEN INTERVAL {Y} DAY PRECEDING AND CURRENT ROW
                    ) AS tx_count_last_Y_days
                FROM {blockchain}.core.fact_transactions t
                LEFT JOIN contract_addresses c ON t.from_address = c.address
                WHERE t.to_address = {contract_address}
                AND t.block_timestamp < {DATE_CUTT_OFF}
                AND c.address IS NULL
            ),
            active_users AS (
                SELECT DISTINCT from_address
                FROM user_transactions
                WHERE tx_count_last_Y_days >= {X}
            )
            SELECT COUNT(*) AS active_users
            FROM active_users;
    """

    # Run the query against Flipside's query engine and await the results
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])


# Farcaster

In [23]:
## Frames

def frames_nusers(name):
    
    
    sql = f"""
            WITH frame_fid AS (
              SELECT fid FROM fnames WHERE username = {name}
            ),

            frame_casts AS (
              SELECT hash, fid
              FROM casts
              WHERE fid = (SELECT fid FROM frame_fid)
                AND timestamp >= NOW() - INTERVAL '{Y} days'
            ),

            frame_reactions AS (
              SELECT fid, COUNT(*) as reaction_count
              FROM reactions
              WHERE target_cast_hash IN (SELECT hash FROM frame_casts)
                AND timestamp >= NOW() - INTERVAL '{Y} days'
              GROUP BY fid
            ),

            frame_replies AS (
              SELECT fid, COUNT(*) as reply_count
              FROM casts
              WHERE parent_hash IN (SELECT hash FROM frame_casts)
                AND timestamp >= NOW() - INTERVAL '{Y} days'
              GROUP BY fid
            ),

            total_engagements AS (
              SELECT COALESCE(r.fid, p.fid) as fid,
                     COALESCE(r.reaction_count, 0) + COALESCE(p.reply_count, 0) as total_count
              FROM frame_reactions r
              FULL OUTER JOIN frame_replies p ON r.fid = p.fid
            )

            SELECT COUNT(*) as active_users
            FROM total_engagements
            WHERE total_count >= {X};

        """
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def frames_interactions(name):
    
    sql = f"""
            WITH frame_fid AS (
              SELECT fid FROM fnames WHERE username = {name}
            ),

            frame_casts AS (
              SELECT hash, fid
              FROM casts
              WHERE fid = (SELECT fid FROM frame_fid)
                AND timestamp >= NOW() - INTERVAL '{Y} days'
            ),

            frame_reactions AS (
              SELECT fid, COUNT(*) as reaction_count
              FROM reactions
              WHERE target_cast_hash IN (SELECT hash FROM frame_casts)
                AND timestamp >= NOW() - INTERVAL '{Y} days'
              GROUP BY fid
            ),

            frame_replies AS (
              SELECT fid, COUNT(*) as reply_count
              FROM casts
              WHERE parent_hash IN (SELECT hash FROM frame_casts)
                AND timestamp >= NOW() - INTERVAL '{Y} days'
              GROUP BY fid
            ),

            total_engagements AS (
              SELECT COALESCE(r.fid, p.fid) as fid,
                     COALESCE(r.reaction_count, 0) + COALESCE(p.reply_count, 0) as total_count
              FROM frame_reactions r
              FULL OUTER JOIN frame_replies p ON r.fid = p.fid
            )

            SELECT 
              (SELECT COUNT(*) FROM frame_casts) +
              (SELECT SUM(reaction_count) FROM frame_reactions) +
              (SELECT SUM(reply_count) FROM frame_replies) as total_transactions;

        """
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])
    
    return()


In [24]:
## Channels

def channel_nusers(channel_url):
    
    sql = f"""
            WITH channel_casts AS (
              SELECT hash, fid
              FROM casts
              WHERE root_parent_url = {channel_url}
                AND timestamp >= NOW() - INTERVAL 'Y days'
            ),

            channel_reactions AS (
              SELECT fid, COUNT(*) as reaction_count
              FROM reactions
              WHERE target_cast_hash IN (SELECT hash FROM channel_casts)
                AND timestamp >= NOW() - INTERVAL '{Y} days'
              GROUP BY fid
            ),

            channel_replies AS (
              SELECT fid, COUNT(*) as reply_count
              FROM casts
              WHERE parent_hash IN (SELECT hash FROM channel_casts)
                AND timestamp >= NOW() - INTERVAL '{Y} days'
              GROUP BY fid
            ),

            total_engagements AS (
              SELECT COALESCE(r.fid, p.fid) as fid,
                     COALESCE(r.reaction_count, 0) + COALESCE(p.reply_count, 0) as total_count
              FROM channel_reactions r
              FULL OUTER JOIN channel_replies p ON r.fid = p.fid
            )

            SELECT COUNT(*) as active_users
            FROM total_engagements
            WHERE total_count >= {X};
    
    """
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])

def channel_interactions(channel_url):
    
    sql = f"""
            WITH channel_casts AS (
              SELECT hash, fid
              FROM casts
              WHERE root_parent_url = {channel_url}
                AND timestamp >= NOW() - INTERVAL '{Y} days'
            ),

            channel_reactions AS (
              SELECT fid, COUNT(*) as reaction_count
              FROM reactions
              WHERE target_cast_hash IN (SELECT hash FROM channel_casts)
                AND timestamp >= NOW() - INTERVAL '{Y} days'
              GROUP BY fid
            ),

            channel_replies AS (
              SELECT fid, COUNT(*) as reply_count
              FROM casts
              WHERE parent_hash IN (SELECT hash FROM channel_casts)
                AND timestamp >= NOW() - INTERVAL '{Y} days'
              GROUP BY fid
            ),

            total_engagements AS (
              SELECT COALESCE(r.fid, p.fid) as fid,
                     COALESCE(r.reaction_count, 0) + COALESCE(p.reply_count, 0) as total_count
              FROM channel_reactions r
              FULL OUTER JOIN channel_replies p ON r.fid = p.fid
            )

            SELECT 
              (SELECT COUNT(*) FROM channel_casts) +
              (SELECT SUM(reaction_count) FROM channel_reactions) +
              (SELECT SUM(reply_count) FROM channel_replies) as total_transactions;

    """
        
    query_result_set = flipside.query(sql)
    
    return(query_result_set.rows[0][0])
